In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import os
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import pearsonr
from pygam import LinearGAM

In [2]:
df = pd.read_parquet("standard_datas.parquet")

In [3]:
df

,InstrumentID_x,TradingDay_x,timestamp,near_price,Volume_x,BidPrice1_x,BidVolume1_x,AskPrice1_x,AskVolume1_x,Turnover_x,...,Turnover_y_z,OpenInterest_x_z,OpenInterest_y_z,BidPrice1_x_z,AskPrice1_x_z,BidPrice1_y_z,AskPrice1_y_z,oi_change_far_z,block_trade_flag_z,vwap_volatility_qn
0,rb2305,2023-01-03,2023-01-03 09:00:06.500,4058.0,22230,4057.0,0.322322,4058.0,0.402402,9.050660e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rb2305,2023-01-03,2023-01-03 09:00:07.000,4058.0,23395,4057.0,0.034535,4058.0,0.064064,9.523320e+08,...,0.707107,-0.707041,-0.706704,0.000000,0.000000,0.707066,0.707087,NaN,-0.707107,NaN
2,rb2305,2023-01-03,2023-01-03 09:00:07.500,4060.0,23832,4058.0,0.505506,4060.0,0.794795,9.700676e+08,...,0.978961,-0.865730,-0.775419,1.154619,1.154660,0.577310,0.000083,NaN,-0.577350,NaN
3,rb2305,2023-01-03,2023-01-03 09:00:08.000,4060.0,24845,4059.0,0.055055,4060.0,0.653153,1.011186e+09,...,0.861769,-1.381095,-0.489559,1.305480,0.865995,1.390652,1.317330,NaN,-0.500000,NaN
4,rb2305,2023-01-03,2023-01-03 09:00:08.500,4060.0,25341,4060.0,0.352352,4061.0,0.238739,1.031323e+09,...,0.784704,-1.026826,-0.438387,1.380388,1.192411,1.499820,1.006154,NaN,-0.447214,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2082603,rb2401,2023-12-07,2023-12-07 22:59:58.500,3941.0,261595,3941.0,0.023524,3942.0,0.160160,1.029894e+10,...,1.967574,-2.447972,-2.195103,-1.729026,-1.807391,-2.177068,-0.962938,-1.799945,0.000000,-0.253471
2082604,rb2401,2023-12-07,2023-12-07 22:59:59.000,3941.0,261604,3941.0,0.035035,3942.0,0.155656,1.029930e+10,...,1.956573,-2.421589,-2.183084,-1.686602,-1.761585,-0.621885,-0.962938,-1.661512,0.000000,-0.262117
2082605,rb2401,2023-12-07,2023-12-07 22:59:59.500,3941.0,261627,3940.0,0.933267,3941.0,0.072573,1.030021e+10,...,1.945428,-2.387996,-2.156170,-3.289191,-3.382717,-2.076589,-2.643503,-1.647445,0.000000,-0.221687
2082606,rb2401,2023-12-07,2023-12-07 23:00:00.000,3941.0,261652,3940.0,0.932933,3941.0,0.031031,1.030119e+10,...,1.936688,-2.416726,-2.137631,-2.975126,-3.046460,-1.986255,-0.881011,-1.636556,0.000000,-0.231779


In [4]:
missing_df = df.isna().sum().reset_index()
missing_df.columns = ['column', 'missing_count']
missing_df = missing_df[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(by='missing_count', ascending=False)


In [5]:
# pd.set_option('display.max_rows', None)

missing_df

,column,missing_count
113,oi_change_far_z,4387
46,oi_change_far,4386
101,oi_change_far_signed_log,4386
115,vwap_volatility_qn,239
103,Volume_x_z,1
104,Volume_y_z,1
105,Turnover_x_z,1
106,Turnover_y_z,1
107,OpenInterest_x_z,1
108,OpenInterest_y_z,1


In [6]:
df['oi_change_far'] = df['oi_change_far'].fillna(0)

df['oi_change_far_signed_log'] = np.sign(df['oi_change_far']) * np.log1p(np.abs(df['oi_change_far']))

roller = df['oi_change_far_signed_log'].rolling(window=480, min_periods=10)
mean = roller.mean()
std = roller.std().replace(0, 1.0)
df['oi_change_far_z'] = (df['oi_change_far_signed_log'] - mean) / std

rolling_q50 = df['vwap_volatility_qn'].rolling(240, min_periods=1).median()
df['vwap_volatility_qn'] = df['vwap_volatility_qn'].fillna(rolling_q50)

z_columns = [col for col in df.columns if col.endswith('_z')]
df[z_columns] = df[z_columns].fillna(method='ffill')

missing_check = df[
    z_columns + ['oi_change_far', 'oi_change_far_signed_log', 'vwap_volatility_qn']
].isnull().sum()

print(missing_check[missing_check > 0])

Volume_x_z              1
Volume_y_z              1
Turnover_x_z            1
Turnover_y_z            1
OpenInterest_x_z        1
OpenInterest_y_z        1
BidPrice1_x_z           1
AskPrice1_x_z           1
BidPrice1_y_z           1
AskPrice1_y_z           1
oi_change_far_z         9
block_trade_flag_z      1
vwap_volatility_qn    239
dtype: int64


/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/415597604.py:19: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[z_columns] = df[z_columns].fillna(method='ffill')


In [7]:
z_columns = [col for col in df.columns if col.endswith('_z')]
df[z_columns] = df[z_columns].fillna(method='ffill')

df['oi_change_far_z'] = df['oi_change_far_z'].fillna(0)

rolling_q50 = df['vwap_volatility_qn'].rolling(240, min_periods=1).median()
df['vwap_volatility_qn'] = df['vwap_volatility_qn'].fillna(rolling_q50)

missing_check = df[
    ['oi_change_far_z', 'vwap_volatility_qn'] + z_columns
].isnull().sum()

print(missing_check[missing_check > 0])


/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/3930856554.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[z_columns] = df[z_columns].fillna(method='ffill')


vwap_volatility_qn    239
Volume_x_z              1
Volume_y_z              1
Turnover_x_z            1
Turnover_y_z            1
OpenInterest_x_z        1
OpenInterest_y_z        1
BidPrice1_x_z           1
AskPrice1_x_z           1
BidPrice1_y_z           1
AskPrice1_y_z           1
block_trade_flag_z      1
dtype: int64


In [67]:
df

,InstrumentID_x,TradingDay_x,near_price,Volume_x,BidPrice1_x,BidVolume1_x,AskPrice1_x,AskVolume1_x,Turnover_x,OpenInterest_x,...,BidPrice1_y_log,AskPrice1_y_log,oi_change_far_signed_log,block_trade_flag_origin,oi_change_far_z,spread_state,book_pressure,vol_momentum,micro_structure,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 09:00:06.500,rb2305,2023-01-03,4058.0,22230,4057.0,0.322322,4058.0,0.402402,9.050660e+08,1929256.0,...,8.303009,8.303505,0.000000,1,0.000000,high,0.958606,NaN,-0.000000,0.499999
2023-01-03 09:00:07.000,rb2305,2023-01-03,4058.0,23395,4057.0,0.034535,4058.0,0.064064,9.523320e+08,1928961.0,...,8.303257,8.304000,0.000000,0,0.000000,high,0.911398,0.000000,-1.786274,0.333333
2023-01-03 09:00:07.500,rb2305,2023-01-03,4060.0,23832,4058.0,0.505506,4060.0,0.794795,9.700676e+08,1928819.0,...,8.303257,8.303752,0.000000,0,0.000000,high,0.971578,0.000000,-1.709319,-0.400000
2023-01-03 09:00:08.000,rb2305,2023-01-03,4060.0,24845,4059.0,0.055055,4060.0,0.653153,1.011186e+09,1928153.0,...,8.303752,8.304495,0.000000,0,0.000000,medium,0.782733,0.000000,-2.366495,1.499995
2023-01-03 09:00:09.000,rb2305,2023-01-03,4060.0,26294,4060.0,0.110611,4061.0,0.030531,1.070009e+09,1927824.0,...,8.304247,8.304742,0.000000,0,0.000000,medium,0.898762,0.000000,-3.313145,1.199996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-07 22:59:58.500,rb2401,2023-12-07,3941.0,261595,3941.0,0.023524,3942.0,0.160160,1.029894e+10,551950.0,...,8.291296,8.291797,-0.000821,0,-1.800011,low,1.618884,-0.000615,-4.734229,0.426087
2023-12-07 22:59:59.000,rb2401,2023-12-07,3941.0,261604,3941.0,0.035035,3942.0,0.155656,1.029930e+10,551947.0,...,8.291547,8.291797,-0.000786,0,-1.661573,low,1.562667,-0.000620,-3.864253,0.423913
2023-12-07 22:59:59.500,rb2401,2023-12-07,3941.0,261627,3940.0,0.933267,3941.0,0.072573,1.030021e+10,551946.0,...,8.291296,8.291547,-0.000784,0,-1.647505,low,1.716226,-0.000621,-3.386653,0.228985


In [10]:
drop_columns = [
    'vwap_volatility_qn', 
    'Volume_x_z', 'Volume_y_z',
    'Turnover_x_z', 'Turnover_y_z',
    'OpenInterest_x_z', 'OpenInterest_y_z',
    'BidPrice1_x_z', 'AskPrice1_x_z',
    'BidPrice1_y_z', 'AskPrice1_y_z',
    'block_trade_flag_z'
]

df = df.drop(columns=drop_columns)

In [63]:
feature_groups = {
    'spread_core': [
        'spread',                  
        'spread_1h_mean',          
        'spread_1h_std'            
    ],
    
    'order_book': [
        'near_bid_ask_spread',     
        'far_bid_ask_spread',      
        'imbalance_diff',          
        'book_pressure'            )
    ],
    
    'volume': [
        'volume_ratio',            
        'turnover_ratio',          
        'open_interest_ratio'      
    ],
    
    'volatility': [
        'spread_1h_std',           
        'vol_curve',               
        'near_far_vol_ratio'       
    ],
    
    'microstructure': [
        'price_pressure_ds',      
        'market_resilience',       
        'tick_imbalance'           
    ],
    
    'temporal': [
        'intraday_time_sin',      
        'intraday_time_cos',       
        'days_to_near_expiry'      
    ],
    
    'signals': [
        'trend_strength_3min_ds',  
        'vol_trigger'              
    ]
}

all_features = [f for group in feature_groups.values() for f in group]

In [64]:
params = {
    'objective': 'quantile',
    'alpha': 0.5,  
    'metric': 'quantile',
    'boosting_type': 'goss',  
    'learning_rate': 0.1,
    'num_leaves': 255,
    'max_depth': 7,
    'min_data_in_leaf': 500,  
    'feature_fraction': 0.7,
    'bagging_freq': 10,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'num_threads': 8,
    'time_out': 600,
    'linear_tree': False,
    'path_smooth': 0.5,
    'extra_trees': True
}

In [ ]:
df = df.set_index('timestamp').sort_index()


In [69]:
forecast_horizon = '1min'  

future_spread = df['spread'].shift(-1).rolling(forecast_horizon).mean()
df['target'] = (future_spread - df['spread']) / (df['spread'].abs() + 1e-6)  

df = df[(df['target'].notnull()) & (df['target'].abs() < 3)] 
df['target'] = df['target'].astype(np.float32)
target = ['target']


/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/1997142204.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'] = df['target'].astype(np.float32)


In [76]:
df['spread_state'] = pd.cut(df['spread'], 
                          bins=np.quantile(df['spread'], [0, 0.3, 0.7, 1]),
                          labels=['low', 'medium', 'high'])

df['volume_ratio'] = df["Volume_x"] / (df["Volume_x"] + df["Volume_y"] + 1e-6)
df['turnover_ratio'] = df["Turnover_x"] / (df["Turnover_x"] + df["Turnover_y"] + 1e-6)
df['open_interest_ratio'] = df["OpenInterest_y"] / (df["OpenInterest_x"] + df["OpenInterest_y"] + 1e-6)
df['near_far_vol_ratio'] = df['near_5min_vol'] / (df['far_5min_vol'] + 1e-6)


df['book_pressure'] = (
    (df['BidVolume1_x'] + df['BidVolume1_y']) / 
    (df['AskVolume1_x'] + df['AskVolume1_y'] + 1e-6)
).rolling('30s').mean()

df['vol_momentum'] = df['vol_curve'].diff().rolling('5min').mean()

df['micro_structure'] = (
    df['price_pressure_ds'] * df['market_resilience']
) / (df['panic_index_ds'] + 1e-6)

vol_window = '24H'       
vol_quantile = 0.75       
min_periods = 1440        

df['spread_1h_std_quantile'] = (
    df['spread_1h_std']
    .shift(1)  
    .rolling(window=vol_window, min_periods=min_periods)
    .quantile(vol_quantile)
)

df['vol_trigger'] = (
    (df['spread_1h_std'] > df['spread_1h_std_quantile'])
    .astype(int)
)

all_features += ['spread_state', 'book_pressure', 'vol_momentum', 'micro_structure']
all_features = [col for col in all_features if pd.api.types.is_numeric_dtype(df[col])]
all_features = list(set(all_features))


/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/106850430.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spread_state'] = pd.cut(df['spread'],
/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/106850430.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['volume_ratio'] = df["Volume_x"] / (df["Volume_x"] + df["Volume_y"] + 1e-6)
/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/106850430.py:8: SettingWithCopyWarning: 
A value is trying to be set on a c

In [77]:
def progressive_feature_selection(df, features, target, params):
    
    train_data = lgb.Dataset(df[features], label=df[target])
    base_model = lgb.train(params, train_data, num_boost_round=500)
    base_importance = pd.DataFrame({
        'feature': features,
        'importance': base_model.feature_importance(importance_type='gain')
    }).sort_values('importance', ascending=False)
    
    phase1_features = base_importance[base_importance['importance'] > 
                      base_importance['importance'].median()]['feature'].tolist()
    print(f"{len(phase1_features)}")
    
    explainer = shap.TreeExplainer(base_model)
    shap_values = explainer.shap_values(df[features])  
    phase1_indices = [features.index(f) for f in phase1_features] 
    selected_shap = shap_values[:, phase1_indices] 
    
    interaction_values = np.abs(selected_shap).mean(axis=0)
    interaction_df = pd.DataFrame({
        'feature': phase1_features,
        'shap_value': interaction_values
    }).sort_values('shap_value', ascending=False)
    
    phase2_features = interaction_df.head(int(len(phase1_features)*0.25))['feature'].tolist()
    print(f"阶段2保留特征数：{len(phase2_features)}")
    
    stability_scores = []
    
    try:
        valid_indices = [base_model.feature_name().index(f) for f in phase2_features]
    except ValueError as e:
        missing = str(e).split()[-1]
        raise ValueError(f"check phase 2")
        
    base_imp = base_model.feature_importance()[valid_indices]

    window_size = pd.Timedelta(hours=1)
    for start_time in pd.date_range(df.index.min(), df.index.max(), freq='6H'):
        end_time = start_time + window_size
        window_data = df[start_time:end_time]
        
        if len(window_data) < 100:
            continue
            
        window_model = lgb.train(params, lgb.Dataset(window_data[phase2_features], window_data[target]))
        window_imp = window_model.feature_importance()
        
        corr = np.corrcoef(base_imp, window_imp)[0,1]
        stability_scores.append(corr)
    
    stable_mask = pd.Series(stability_scores).rolling(5).mean().fillna(0) > 0.6
    final_features = interaction_df[stable_mask]['feature'].tolist()
    
    print(f"{len(final_features)}")
    return final_features

selected_features = progressive_feature_selection(df, all_features, target, params)

[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Warning] Found boosting=goss. For backwards compatibility reasons, LightGBM interprets this as boosting=gbdt, data_sample_strategy=goss.To suppress this warning, set data_sample_strategy=goss instead.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016345 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4585
[LightGBM] [Info] Number of data points in the train set: 1857410, number of used features: 22
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/2340206458.py:50: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for start_time in pd.date_range(df.index.min(), df.index.max(), freq='6H'):


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, 

/var/folders/62/pm6rwllx53d08gq5xbp3_njr0000gn/T/ipykernel_8944/2340206458.py:67: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  final_features = interaction_df[stable_mask]['feature'].tolist()


In [80]:
selected_features

['turnover_ratio', 'vol_curve', 'tick_imbalance']

In [81]:
df

,InstrumentID_x,TradingDay_x,near_price,Volume_x,BidPrice1_x,BidVolume1_x,AskPrice1_x,AskVolume1_x,Turnover_x,OpenInterest_x,...,spread_state,book_pressure,vol_momentum,micro_structure,target,turnover_ratio,open_interest_ratio,near_far_vol_ratio,spread_1h_std_quantile,vol_trigger
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-03 09:00:06.500,rb2305,2023-01-03,4058.0,22230,4057.0,0.322322,4058.0,0.402402,9.050660e+08,1929256.0,...,high,0.958606,NaN,-0.000000,0.499999,0.986404,0.127638,1.630931,NaN,0
2023-01-03 09:00:07.000,rb2305,2023-01-03,4058.0,23395,4057.0,0.034535,4058.0,0.064064,9.523320e+08,1928961.0,...,high,0.911398,0.000000,-1.786274,0.333333,0.986574,0.127652,1.629391,NaN,0
2023-01-03 09:00:07.500,rb2305,2023-01-03,4060.0,23832,4058.0,0.505506,4060.0,0.794795,9.700676e+08,1928819.0,...,high,0.971578,0.000000,-1.709319,-0.400000,0.986371,0.127660,1.668266,NaN,0
2023-01-03 09:00:08.000,rb2305,2023-01-03,4060.0,24845,4059.0,0.055055,4060.0,0.653153,1.011186e+09,1928153.0,...,medium,0.782733,0.000000,-2.366495,1.499995,0.986840,0.127699,1.306651,NaN,0
2023-01-03 09:00:09.000,rb2305,2023-01-03,4060.0,26294,4060.0,0.110611,4061.0,0.030531,1.070009e+09,1927824.0,...,medium,0.898762,0.000000,-3.313145,1.199996,0.987334,0.127717,1.288401,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-07 22:59:58.500,rb2401,2023-12-07,3941.0,261595,3941.0,0.023524,3942.0,0.160160,1.029894e+10,551950.0,...,low,1.618884,-0.000615,-4.734229,0.426087,0.340847,0.691619,0.566576,0.581565,0
2023-12-07 22:59:59.000,rb2401,2023-12-07,3941.0,261604,3941.0,0.035035,3942.0,0.155656,1.029930e+10,551947.0,...,low,1.562667,-0.000620,-3.864253,0.423913,0.340849,0.691618,0.549624,0.581565,0
2023-12-07 22:59:59.500,rb2401,2023-12-07,3941.0,261627,3940.0,0.933267,3941.0,0.072573,1.030021e+10,551946.0,...,low,1.716226,-0.000621,-3.386653,0.228985,0.340864,0.691619,0.566204,0.581565,0


In [90]:
feature_df = df[selected_features + ['spread'] + ['near_price'] + ['far_price']].copy()
feature_df = feature_df.reset_index()

feature_df.to_parquet(
    'arbitrage_features.parquet',
    engine='pyarrow',
    compression='ZSTD',  
    index=False,          
    coerce_timestamps='ms',  
    allow_truncated_timestamps=False
)

In [91]:
print(f"{feature_df.columns.tolist()}")
print(f"{feature_df['timestamp'].min()} 至 {feature_df['timestamp'].max()}")
print(f"{os.path.getsize('arbitrage_features.parquet')/1024/1024:.2f} MB")

In [92]:
pd.read_parquet('arbitrage_features.parquet')

,timestamp,turnover_ratio,vol_curve,tick_imbalance,spread,near_price,far_price
0,2023-01-03 09:00:06.500,0.986404,-1.402211,0.308308,0.666667,4058.0,4037.0
1,2023-01-03 09:00:07.000,0.986574,-1.402211,0.308308,1.000000,4058.0,4036.0
2,2023-01-03 09:00:07.500,0.986371,-1.402211,0.333834,1.666667,4060.0,4036.0
3,2023-01-03 09:00:08.000,0.986840,-1.402211,0.333834,0.333333,4060.0,4040.0
4,2023-01-03 09:00:09.000,0.987334,-1.402211,0.333834,0.333333,4060.0,4040.0
...,...,...,...,...,...,...,...
1857405,2023-12-07 22:59:58.500,0.340847,-0.623299,0.094094,-1.000000,3941.0,3990.0
1857406,2023-12-07 22:59:59.000,0.340849,-0.625581,0.094094,-1.000000,3941.0,3990.0
1857407,2023-12-07 22:59:59.500,0.340864,-0.625662,0.089089,-0.750000,3941.0,3989.0
1857408,2023-12-07 23:00:00.000,0.340879,-0.627668,0.089089,-1.000000,3941.0,3990.0
